# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 22 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 23). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la energía generada 24 horas después. 

Por lo tanto, vamos a comenzar con todos los imports necesarios para este proyecto

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import PowerTransformer



También, vamos a importar el documento con los datos de entrenamiento, y le vamos a eliminar las columnas innecesarias

In [2]:
import pandas as pd


# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# FIlter the data to only include the columns that end in 13
data = data.filter(regex='13$|energy')
#print(data)



Y separar los datos de entrenamiento de los datos de test, con train_test_split

In [3]:
from sklearn.model_selection import train_test_split

y = data['energy']
x = data.drop(columns=['energy'])

# test_size vamos a escoger 0.2 ya que tenemos datos en un rango de 5 años en orden (2015-2019) por lo que (asumiendo que el numero de datos para todos los años es similar) usaremos 2019 como test final.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)
#print(f"Fechas de tain {train.iloc[0].datetime}-{train-iloc[-1].datetime}")

In [4]:
# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  
from sklearn.pipeline import Pipeline


scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer(), PowerTransformer]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])



# Decidir usando KNN
A continuación, usaremos KNN para determinar el método de escalado más apropiado para el problema. 
Dado que no queremos entrenar con datos en el futuro, no podremos usar una validacin cruzada normal (usando Kfold o Shuffle), por lo que tendremos que usar predefined split o Time Series split (el cual es igual a una validación cruzada, pero ignora los posteriores a test), en nuestro caso, probaremos con.......................................................................................................................................................................................................................


In [7]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
# Usaremos los parametros por default n_slits = 5, max_train_dize = None, test_size = None, gap = 0 
cv = TimeSeriesSplit()

param_grid = {
    'n_neighbors': [5, 10, 15, 20], 
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [30, 40, 50],
    'p' : [2], # distance `= 1 manhatan, p=2 euclidean
}

model = GridSearchCV(KNeighborsRegressor,
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')
print(model)
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])
print(pipe)
pipe.fit(X_train, y_train)

best_model = model.best_estimator_
best_params = model.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)


TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

# Entrenamiento de diferentes modelos
Lo próximo que haremos, será probar con diferentes métodos de entrenamiento (KNN, árboles de regresión, regresión lineal normal y la variante de Lasso, y SVM. Para elegir el mejor, usaremos la librería time para saber cuánto se tarda en entrenar cada modelo, un dummy para comparar resultados, y el mejor de entre ellos, usando una política de importancia de 20% a tiempo y 80% precisión.

Lo primero que vamos a usar es el dummyRegressor, para tener una representación visual del peor escenario posible y que tanto mejoran nuestros diseños.

In [14]:
# Create a Dunny regressor to obtain a baseline

from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', dummy)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

rmse_dummy = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_dummy = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_dummy}")
print(f"SQRT RMSE: {sqrt_rmse_dummy}")



RMSE: 451489.08466268575
SQRT RMSE: 671.9293747580067


Para empezar, vamos a comparar con un Decision Tree Regresor.

In [ ]:
# Optimizing hyperparameters
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor instead of DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
import numpy as np
import time


param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 12, 13, 14, 15, 16, 17],
    'min_samples_split': range(2, 15, 2),
    'min_samples_leaf': range(5, 15, 2),
}

cv = TimeSeriesSplit()
model = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error', n_iter=50)  # Adjust scoring metric

model2 = GridSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


rmse_tree = metrics.mean_squared_error(y_test, y_pred)
sqrt_rmse_tree = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse_tree}")
print(f"SQRT RMSE: {sqrt_rmse_tree}")
print(f"Best params: {model.best_params_}")
# Best params: {'criterion': 'absolute_error', 'max_depth': 15, 'min_samples_leaf': 11, 'min_samples_split': 3}


Como hemos podido observar, el modelo es bastante mejor que el dummy, aproximadamente, un tercio mejor. No obstante probaremos con más modelos para contrastar.

El próximo modelo es la regresión lineal normal

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV


param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1],  # -1 indica utilizar todos los núcleos
    'positive': [True, False]
}


# Crear el modelo de regresión lineal
model = LinearRegression()
cv = TimeSeriesSplit()

grid_search = GridSearchCV(model,
                           param_grid,
                           cv=cv,
                           scoring="neg_mean_absolute_percentage_error"
                           )

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)



C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=1.82501e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.06611e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.24549e-17): result may not be accurate.
  s[P] = solve(AtA[P_ind[:, None], P_ind[None, :]], Atb[P],
C:\Users\Administrador\OneDrive\Escritorio\Uni\Tercero Uni\AA-LAB1\venv\lib\site-packages\scipy\optimize\_nnls.py:135: LinAlgWarning: Ill-conditioned matrix (rcond=2.33915e-17): result may not be accura

Mejor modelo: LinearRegression(n_jobs=-1)
Mejorres parámetros: {'copy_X': True, 'fit_intercept': True, 'n_jobs': -1, 'positive': False}
RMSE: 559.7095305382695


Como podemos observar, aunque es mejor que el dummy, está bastante peor que el árbol de decisión, no obstante, vamos a probar con  con la variante de Lasso a ver si conseguimos mejorar el entrenamiento:

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

# Definir los parámetros que deseas probar
parameters = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Valores de regularización
    'fit_intercept': [True, False],
    'copy_X': [True, False], # pendientes de borrar
    'positive': [True, False], 
    'precompute': [True, False], 
    'random_state': [7543], 
    'selection': ['cyclic', 'random'], 
    'tol': [1**(-10), 1**(-5), 1**(-4), 1**(-3), 1**(-2)], 
    'warm_start': [True, False]
}

# Crear el modelo de regresión lineal Lasso
model = Lasso()
cv = TimeSeriesSplit()
grid_search = GridSearchCV(model, 
                           parameters, 
                           scoring='neg_mean_squared_error', 
                           cv=cv)

grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)

Mejor modelo: Lasso(alpha=0.1, precompute=True, random_state=7543, selection='random',
      tol=1.0, warm_start=True)
Mejorres parámetros: {'alpha': 0.1, 'copy_X': True, 'fit_intercept': True, 'positive': False, 'precompute': True, 'random_state': 7543, 'selection': 'random', 'tol': 1.0, 'warm_start': True}
RMSE: 567.7651275011757


Afortunadamente, el modelo es mejor que la regresión lineal normal, pero los árboles le sacan bastante más precisión (aunque tardan más en entrenar, el beneficio sobrepasa los costes.


In [31]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
# Usaremos los parametros por default n_slits = 5, max_train_dize = None, test_size = None, gap = 0 
cv = TimeSeriesSplit()

param_grid = {
    'n_neighbors': range(7, 15, 1), 
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(20, 40, 5),
    'p' : [2], # distance `= 1 manhatan, p=2 euclidean
}

model = KNeighborsRegressor()
grid_search = GridSearchCV(model,
                     param_grid, cv=cv, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = grid_search.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mejor modelo:",best_model)
print("Mejorres parámetros:",best_params)
print("RMSE:", rmse)


Mejor modelo: KNeighborsRegressor(leaf_size=20, n_neighbors=12)
Mejorres parámetros: {'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 12, 'p': 2, 'weights': 'uniform'}
RMSE: 601.0721146249629


Los últimos modelos que vamos a entrenar van a ser KNN, SVM y redes de neuronas

## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.